# Cost savings calculations

In [9]:
import json


projects = ['streamstone', 'orleans','identityazuretable', 'insights', 'durabletask']

*Helper functions*

In [10]:
# extract selected tests (tests interacting with the cloud) and discrepant tests
def get_methods_from_file(file_path):

    with open(file_path, "r") as f:
        methods = f.read().splitlines()

    return methods

# open hash file
def read_hash_file(project, commit_number):
    
    with open(f"{project}/hashes_{commit_number}.json", 'r') as f:
        return json.load(f)

*Identify discrepant and selected tests*

In [11]:
def find_tests_with_methods(methods_json_path, desc_methods):

    with open(methods_json_path, "r") as f:
        json_data = json.load(f)

    cloud_tests = []

    for i in json_data.keys():
        if not json_data[i] == []: # remove tests with no cloud APIs
          cloud_tests.append(i)

    tests_with_desc_methods = []
    for key, value in json_data.items():
        for method in desc_methods:
            if method in value:
                if not key in tests_with_desc_methods:
                    tests_with_desc_methods.append(key)

    return cloud_tests, tests_with_desc_methods


methods_file_path = "discrepant_apis.txt"
methods = get_methods_from_file(methods_file_path)
discrepant_test = {}   # tests which make discrepant api calls
selected_test = {}     # selected tests

for j in projects:

    methods_json_path = f"projects_api_calls/{j}.json"

    all_tests, d_tests = find_tests_with_methods(methods_json_path, methods)

    discrepant_test[j] = d_tests
    selected_test[j] = all_tests

## Total requests per project

In [12]:
# sum all apis by tests for all commits per project
def sum_tests_requests(project):
    
    sum_per_commit = {}
    for commit_number in range(5, 0, -1):
        hash_file = read_hash_file(project, commit_number)
        sum_per_commit[commit_number] = 0
        for test in hash_file:
            sum_per_commit[commit_number] += len(hash_file[test])

    return sum_per_commit

total_per_commit = {}
for project in projects:
    total_per_commit[project] = sum_tests_requests(project)


avg_total_per_commit = {key: sum(value[commit] for commit in range(1, 6)) / 5 for key, value in total_per_commit.items()}

print("Average total number of requests")
print(json.dumps(avg_total_per_commit, indent=4))

Average total number of requests
{
    "streamstone": 590.0,
    "orleans": 117904.8,
    "identityazuretable": 9858.4,
    "insights": 5249.0,
    "durabletask": 79654.4
}


## Policy 1

In [13]:
cost_policy1 = {}

for project in projects:
    cost_policy1[project] = {}
    for commit_number in range(5, 0, -1):
        hash_file = read_hash_file(project, commit_number)
        cost_policy1[project][commit_number] = {'cost' : 0}
        for test in hash_file:
            if test in discrepant_test[project]:
                cost_policy1[project][commit_number]['cost'] += len(hash_file[test])

avg_total_per_commit = {key: sum(value[commit] for commit in range(1, 5)) / 4 for key, value in total_per_commit.items()}
avg_cost_policy1 = {key: 100*(sum(total_per_commit[key][commit] - value[commit]['cost'] for commit in range(1, 5)) / 4) / avg_total_per_commit[key] for key, value in cost_policy1.items()}

print('The average cost of policy 1 is:')
print(json.dumps(avg_cost_policy1, indent=2))

The average cost of policy 1 is:
{
  "streamstone": 0.0,
  "orleans": 29.411476035939156,
  "identityazuretable": 100.0,
  "insights": 4.29000429000429,
  "durabletask": 0.0
}


## Policy 2

In [14]:
# calculate policy 2 cost

cost_policy2 = {}
track_policy2_discrepant_tests = {}

# populate 5th commit
for project in projects:
    cost_policy2[project] = {5: {'cost' : 0}}
    hash_file = read_hash_file(project, 5)
    for test in hash_file:
        cost_policy2[project][5]['cost'] += len(hash_file[test])
        
for project in projects:
    track_policy2_discrepant_tests[project] = {}
    for commit_number in range(4, 0, -1):
        hash_file1 = read_hash_file(project, commit_number+1)
        hash_file2 = read_hash_file(project, commit_number)
        cost_policy2[project][commit_number] = {'cost' : 0}
        track_policy2_discrepant_tests[project][commit_number] = []
        for test in hash_file2:
            if test in hash_file1 and hash_file1[test] != hash_file2[test]:
                track_policy2_discrepant_tests[project][commit_number].append(test)
                cost_policy2[project][commit_number]['cost'] += len(hash_file2[test])
            # Add discrepant test result cost
            if project == 'streamstone' and test == 'When_disabled':
                cost_policy2[project][commit_number]['cost'] += len(hash_file2[test])

avg_total_per_commit = {key: sum(value[commit] for commit in range(1, 5)) / 4 for key, value in total_per_commit.items()}
avg_cost_policy2 = {key: 100*(sum(total_per_commit[key][commit] - value[commit]['cost'] for commit in range(1, 5)) / 4) / avg_total_per_commit[key] for key, value in cost_policy2.items()}

print('The average cost of policy 2 is:')
print(json.dumps(avg_cost_policy2, indent=2))

The average cost of policy 2 is:
{
  "streamstone": 99.15254237288136,
  "orleans": 0.3548255636563035,
  "identityazuretable": 0.3650005069451485,
  "insights": 47.19004719004719,
  "durabletask": 0.0
}


## Policy 3

In [15]:
cost_policy3 = {}

for project in projects:
    cost_policy3[project] = {}
    for commit_number in range(5, 0, -1):
        hash_file = read_hash_file(project, commit_number)
        cost_policy3[project][commit_number] = {'cost' : 0}
        if commit_number != 5:
            for test in hash_file:
                if test in discrepant_test[project] and test in track_policy2_discrepant_tests[project][commit_number]:
                    cost_policy3[project][commit_number]['cost'] += len(hash_file[test])
                # Add discrepant test result cost
                if project == 'streamstone' and test == 'When_disabled':
                    cost_policy3[project][commit_number]['cost'] += len(hash_file[test])

avg_total_per_commit = {key: sum(value[commit] for commit in range(1, 5)) / 4 for key, value in total_per_commit.items()}
avg_cost_policy3 = {key: 100*(sum(total_per_commit[key][commit] - value[commit]['cost'] for commit in range(1, 5)) / 4) / avg_total_per_commit[key] for key, value in cost_policy3.items()}

print('The average cost of policy 3 is:')
print(json.dumps(avg_cost_policy3, indent=2))

The average cost of policy 3 is:
{
  "streamstone": 99.15254237288136,
  "orleans": 29.57192632088777,
  "identityazuretable": 100.0,
  "insights": 47.19004719004719,
  "durabletask": 0.0
}
